# Initialization

In [1]:
import numpy as np
import pandas as pd
import time
import io

In [2]:
tic = time.time()

# Load Data

In [3]:
df = pd.read_csv('PreprocessingData.csv')

# Data Understanding

In [4]:
df.head()

,user_id,gender,city,payment_amount,payment_date
0,200806,زن,تهران,1540000,2017-09-09
1,200806,زن,تهران,1510000,2017-09-01
2,11156,مرد,اصفهان,43200000,2016-05-21
3,200806,زن,تهران,1260000,2017-07-18
4,245638,مرد,تهران,1240000,2018-02-02


In [5]:
df.drop('user_id', axis=1).describe()

,payment_amount
count,1.490804e+06
mean,9.667401e+05
std,9.343177e+06
min,-2.147484e+09
25%,2.000000e+04
50%,1.500000e+05
75%,8.481000e+05
max,1.761294e+09


## Show is null

In [6]:
df.isnull().sum()

user_id             0
gender            951
city                0
payment_amount      0
payment_date        0
dtype: int64

# Data Cleaning

## Remove negative amount

In [7]:
clean_pdf = df[df.payment_amount > 0]

# Change type of date column and create recency column

In [8]:
max_payment_date = clean_pdf.payment_date.max()
min_payment_date = clean_pdf.payment_date.min()

In [9]:
clean_pdf.loc[:, 'transaction_recency'] = pd.to_datetime(max_payment_date) - pd.to_datetime(clean_pdf.payment_date)

C:\Users\azimi\AppData\Local\Temp\ipykernel_14120\2769625300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_pdf.loc[:, 'transaction_recency'] = pd.to_datetime(max_payment_date) - pd.to_datetime(clean_pdf.payment_date)


In [10]:
max_payment_date

'2018-09-08'

In [11]:
week_series = pd.period_range(start=min_payment_date, end='2018-10-08', freq='W-FRI')

In [12]:
max_payment_date

'2018-09-08'

In [13]:
temp_df = pd.DataFrame(data={'start_time': week_series.start_time, 'end_time': week_series.end_time })

In [14]:
temp_df.loc[:, 'new_end_time'] = temp_df.end_time.shift(periods=-3)

In [15]:
temp_df = temp_df.dropna()

In [16]:
temp_df.head(10)

,start_time,end_time,new_end_time
0,2016-03-19,2016-03-25 23:59:59.999999999,2016-04-15 23:59:59.999999999
1,2016-03-26,2016-04-01 23:59:59.999999999,2016-04-22 23:59:59.999999999
2,2016-04-02,2016-04-08 23:59:59.999999999,2016-04-29 23:59:59.999999999
3,2016-04-09,2016-04-15 23:59:59.999999999,2016-05-06 23:59:59.999999999
4,2016-04-16,2016-04-22 23:59:59.999999999,2016-05-13 23:59:59.999999999
5,2016-04-23,2016-04-29 23:59:59.999999999,2016-05-20 23:59:59.999999999
6,2016-04-30,2016-05-06 23:59:59.999999999,2016-05-27 23:59:59.999999999
7,2016-05-07,2016-05-13 23:59:59.999999999,2016-06-03 23:59:59.999999999
8,2016-05-14,2016-05-20 23:59:59.999999999,2016-06-10 23:59:59.999999999
9,2016-05-21,2016-05-27 23:59:59.999999999,2016-06-17 23:59:59.999999999


In [17]:
temp_df.loc[:, 'week_no'] = temp_df.index

In [18]:
temp_df[(temp_df.start_time < '2017-07-26') & (temp_df.new_end_time > '2017-07-26')]

,start_time,end_time,new_end_time,week_no
67,2017-07-01,2017-07-07 23:59:59.999999999,2017-07-28 23:59:59.999999999,67
68,2017-07-08,2017-07-14 23:59:59.999999999,2017-08-04 23:59:59.999999999,68
69,2017-07-15,2017-07-21 23:59:59.999999999,2017-08-11 23:59:59.999999999,69
70,2017-07-22,2017-07-28 23:59:59.999999999,2017-08-18 23:59:59.999999999,70


In [56]:
def get_period_number(date):
    num = temp_df[(temp_df.start_time < date) & (temp_df.new_end_time > date)]
    return num['week_no'].values.tolist()

In [57]:
print(get_period_number('2017-07-26'))

[67, 68, 69, 70]


In [59]:
# test_pdf = clean_pdf.loc[1:10, :]
# test_pdf

,user_id,gender,city,payment_amount,payment_date,transaction_recency
1,200806,زن,تهران,1510000,2017-09-01,372 days
2,11156,مرد,اصفهان,43200000,2016-05-21,840 days
3,200806,زن,تهران,1260000,2017-07-18,417 days
4,245638,مرد,تهران,1240000,2018-02-02,218 days
5,239954,مرد,اصفهان,1030000,2018-02-18,202 days
6,22879,مرد,هرمزگان,129622500,2018-02-06,214 days
7,8906,زن,تهران,93994000,2018-05-09,122 days
8,91728,مرد,البرز,900000,2017-04-23,503 days
9,200806,زن,تهران,900000,2017-05-27,469 days
10,22879,مرد,هرمزگان,160397000,2018-05-30,101 days


In [ ]:
clean_pdf.loc[:, 'week_no'] = clean_pdf.payment_date.apply(get_period_number)

In [60]:
# test_pdf.loc[:, 'week_no'] = test_pdf.payment_date.apply(get_period_number)

C:\Users\azimi\AppData\Local\Temp\ipykernel_14120\2891464066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_pdf.loc[:, 'week_no'] = test_pdf.payment_date.apply(get_period_number)


In [61]:
test_pdf

,user_id,gender,city,payment_amount,payment_date,transaction_recency,week_no
1,200806,زن,تهران,1510000,2017-09-01,372 days,"[72, 73, 74, 75]"
2,11156,مرد,اصفهان,43200000,2016-05-21,840 days,"[6, 7, 8]"
3,200806,زن,تهران,1260000,2017-07-18,417 days,"[66, 67, 68, 69]"
4,245638,مرد,تهران,1240000,2018-02-02,218 days,"[94, 95, 96, 97]"
5,239954,مرد,اصفهان,1030000,2018-02-18,202 days,"[97, 98, 99, 100]"
6,22879,مرد,هرمزگان,129622500,2018-02-06,214 days,"[95, 96, 97, 98]"
7,8906,زن,تهران,93994000,2018-05-09,122 days,"[108, 109, 110, 111]"
8,91728,مرد,البرز,900000,2017-04-23,503 days,"[54, 55, 56, 57]"
9,200806,زن,تهران,900000,2017-05-27,469 days,"[59, 60, 61]"
10,22879,مرد,هرمزگان,160397000,2018-05-30,101 days,"[111, 112, 113, 114]"


# RFM Analysis

In [24]:
rfm_df = clean_pdf.groupby("user_id").agg(
    R=pd.NamedAgg(column="transaction_recency", aggfunc="min"),
    F=pd.NamedAgg(column="user_id", aggfunc="count"),
    M=pd.NamedAgg(column="payment_amount", aggfunc="sum"),
)

In [25]:
import os

In [26]:
os.path.isdir(r'\RFM')

False

In [27]:
rfm_df.to_parquet(path=r'\RFM')

In [28]:
print(time.time() - tic)

2.7979016304016113
